In [ ]:
!pip install -U langchain-community

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 98.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [ ]:
!pip install -U bitsandbytes transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 116.3 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.3
    Uninstalling transformers-4.48.3:
      Successfully uninstalled transformers-4.48.3


In [ ]:
!pip install faiss-gpu-cu12

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 MB 45.0 MB/s eta 0:00:00


In [ ]:
!pip install -U langchain-community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 88.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.5 MB/s eta 0:00:00


In [1]:
import pandas as pd
import numpy as np
import torch
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

In [2]:
data_path = "/content/drive/MyDrive/"

In [3]:
train = pd.read_csv(data_path+"train.csv" )
test = pd.read_csv(data_path+"test.csv" )
sample = pd.read_csv(data_path+"sample_submission.csv")

In [4]:
def preprocess(df):
    df.replace('-', np.nan, inplace=True)
    df['공사종류(대분류)'] = df['공사종류'].str.split(' / ').str[0]
    df['공사종류(중분류)'] = df['공사종류'].str.split(' / ').str[1]
    df['공종(대분류)'] = df['공종'].str.split(' > ').str[0]
    df['공종(중분류)'] = df['공종'].str.split(' > ').str[1]
    df['사고객체(대분류)'] = df['사고객체'].str.split(' > ').str[0]
    df['사고객체(중분류)'] = df['사고객체'].str.split(' > ').str[1]
    df['사고인지 시간'] = df['사고인지 시간'].str.split('-').str[0].str.strip()

    return df


train = preprocess(train)
test = preprocess(test)


In [5]:
# 훈련 데이터 통합 생성
combined_training_data = train.apply(
    lambda row: {
        "question": (
            f"작업 프로세스는 '{row['작업프로세스']}'이며 {row['인적사고']}발생, 사고 원인은 '{row['사고원인']}'입니다."
            f"재발 방지 대책 및 향후 조치 계획은 무엇인가요?"
        ),
        "answer": row["재발방지대책 및 향후조치계획"]
    },
    axis=1
)

# DataFrame으로 변환
combined_training_data = pd.DataFrame(list(combined_training_data))

In [6]:
# 훈련 데이터 통합 생성
combined_test_data = test.apply(
    lambda row: {
        "question": (
            f"작업 프로세스는 '{row['작업프로세스']}'이며 {row['인적사고']}발생, 사고 원인은 '{row['사고원인']}'입니다."
            f"재발 방지 대책 및 향후 조치 계획은 무엇인가요?"
        )
    },
    axis=1
)

# DataFrame으로 변환
combined_test_data = pd.DataFrame(list(combined_test_data))

In [7]:
model_id = "UNIVA-Bllossom/DeepSeek-llama3.1-Bllossom-8B"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    #quantization_config=bnb_config,
    device_map="auto"
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model-00001-of-00004.safetensors:   2%|2         | 115M/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [8]:
import torch
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_community.llms import HuggingFacePipeline
from transformers import pipeline
from tqdm import tqdm

# Train 데이터 준비
train_questions_prevention = combined_training_data['question'].tolist()
train_answers_prevention = combined_training_data['answer'].tolist()

train_documents = [
    f"Q: {q1}\nA: {a1}"
    for q1, a1 in zip(train_questions_prevention, train_answers_prevention)
]

# 임베딩 생성
embedding_model_name = "intfloat/multilingual-e5-large-instruct"
embedding = HuggingFaceEmbeddings(model_name=embedding_model_name)

# 벡터 스토어에 문서 추가
vector_store = FAISS.from_texts(train_documents, embedding)

# Retriever 정의
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 1})






<ipython-input-8-b347abcfbb60>:21: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name=embedding_model_name)
Device set to use cuda:0
<ipython-input-8-b347abcfbb60>:54: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=text_generation_pipeline)


In [26]:
text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    do_sample=True,
    temperature=0.1, # 0~1 까지 0에 가까울 수록 보수적인 답변 즉 창의성이 떨어짐
    return_full_text=False,
    max_new_tokens=512, # 문장 최대 길이 조정
    batch_size=16  # 배치 크기 지정
)

Device set to use cuda:0


In [27]:
prompt_template = """
### 지침: 당신은 건설 안전 전문가입니다.
질문에 대한 답변을 핵심 내용만 요약하여 간략하게 작성하세요.
- 서론, 배경 설명 또는 추가 설명을 절대 포함하지 마세요.
- 다음과 같은 조치를 취할 것을 제안합니다: 와 같은 내용을 포함하지 마세요.
{context}

### 질문:
{question}

[/INST]
"""

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# 커스텀 프롬프트 생성
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

# RAG 체인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}
)

In [28]:
# TensorFloat32 경고 해결: Matmul 성능 최적화
from datasets import Dataset
torch.set_float32_matmul_precision('high')

# 테스트 데이터 준비 (Dataset 사용)
test_questions = combined_test_data['question'].tolist()
test_dataset = Dataset.from_dict({"question": test_questions})
batch_size = 16  # GPU 메모리에 맞게 조정

test_results = []
print("테스트 실행 시작... 총 테스트 샘플 수:", len(test_questions))

# 배치 단위로 처리 (Dataset 활용)
def process_batch(batch):
    # 배치 내 질문들에 대해 리트리버로 컨텍스트 검색
    contexts = [retriever.invoke(q)[0].page_content for q in batch["question"]]
    # 프롬프트 생성
    prompts = [prompt.format(context=c, question=q) for c, q in zip(contexts, batch["question"])]
    # 배치로 파이프라인 실행
    results = text_generation_pipeline(prompts, batch_size=batch_size)
    return [r[0]["generated_text"] for r in results]

# DataLoader처럼 배치 처리
for i in tqdm(range(0, len(test_dataset), batch_size), desc="처리 진행률"):
    batch = test_dataset[i:i + batch_size]  # Dataset 슬라이싱
    batch_results = process_batch(batch)
    test_results.extend(batch_results)

print("\n테스트 실행 완료! 총 결과 수:", len(test_results))

테스트 실행 시작... 총 테스트 샘플 수: 964


처리 진행률: 100%|██████████| 61/61 [36:43<00:00, 36.13s/it]


테스트 실행 완료! 총 결과 수: 964


In [29]:
test_results

['### 답변:\n**요약:**\n재발 방지 대책 및 향후 조치 계획은 다음과 같습니다:\n1. **관리적 대책:** 차량계 건설기계 작업 계획서 작성 및 검토, 펌프카 안전 점검 체크리스트 사전 점검.\n2. **기술적 대책:** 아웃트리거 설치 시 지반 상태 사전 점검 및 받침대 추가 설치 확인, 유도자/신호수 배치, 작업구역 설정, 타 근로자 출입금지 조치.\n3. **교육적 대책:** 펌프카 작업 전 안전 교육 실시 및 확인, 작업 전 TBM 작업 내용 및 위험 포인트 공유.\n4. **향후 조치 계획:** 재발 방지 대책 수립 및 현장 관리 철저.\n\n**최종 답변:**\n재발 방지 대책 및 향후 조치 계획은 다음과 같습니다:\n1. **관리적 대책:** 차량계 건설기계 작업 계획서 작성 및 검토, 펌프카 안전 점검 체크리스트 사전 점검.\n2. **기술적 대책:** 아웃트리거 설치 시 지반 상태 사전 점검 및 받침대 추가 설치 확인, 유도자/신호수 배치, 작업구역 설정, 타 근로자 출입금지 조치.\n3. **교육적 대책:** 펌프카 작업 전 안전 교육 실시 및 확인, 작업 전 TBM 작업 내용 및 위험 포인트 공유.\n4. **향후 조치 계획:** 재발 방지 대책 수립 및 현장 관리 철저.\n\n**최종 답변:**\n재발 방지 대책 및 향후 조치 계획은 다음과 같습니다:\n1. **관리적 대책:** 차량계 건설기계 작업 계획서 작성 및 검토, 펌프카 안전 점검 체크리스트 사전 점검.\n2. **기술적 대책:** 아웃트리거 설치 시 지반 상태 사전 점검 및 받침대 추가 설치 확인, 유도자/신호수 배치, 작업구역 설정, 타 근로자 출입금지 조치.\n3. **교육적 대책:** 펌프카 작업 전 안전 교육 실시 및 확인, 작업 전 TBM 작업 내용 및 위험 포인트 공유.\n4. **향후 조치 계획:**',
 '### 답변:\n**요약:**\n1. **교육 강화**: 작업자에게 안전 교육을 강화하여 절단 작업의 위험성과 안전 조치 방법을 이해하도록 합니다.

In [21]:
test_results[0]

'### 답변:\n**핵심 내용 요약:**\n- **재발 방지 대책:**\n  1. **차량계 건설기계 작업 계획서 작성 및 검토:** 작업 계획을 세우고 점검.\n  2. **펌프카 안전 점검 체크리스트 사전 점검:** 안전을 보장.\n  3. **지반 상태 사전 점검 및 받침대 추가 설치 확인:** 지반 안정성 확보.\n  4. **유도자/신호수 배치:** 작업 공간 관리.\n  5. **타 근로자 출입금지 조치:** 작업자 안전.\n  6. **펌프카 작업 전 안전 교육 및 확인:** 작업자 교육.\n  7. **TBM 작업 내용 및 위험 포인트 공유:** 작업 내부 통신.\n- **향후 조치 계획:**\n  1. **재발 방지 대책 수립 및 현장 관리 철저:** 지속적인 개선.\n  2. **교육 및 훈련 강화:** 지속적인 안전 교육.\n\n**최종 답변:**\n재발 방지 대책으로 차량'

In [25]:
import re

def extract_clean_prevention_measures(text):
    # "재발 방지 대책" 부분만 추출
    pattern = r"재발 방지 대책[:\n]+([\s\S]*?)(?=\n-?\s*향후 조치 계획|$)"
    match = re.search(pattern, text)

    if not match:
        return ""

    extracted_text = match.group(1).strip()

    # **, - 등 특수기호 제거
    cleaned_text = re.sub(r"[\*\-]", "", extracted_text)

    # 숫자로 나열된 부분 제거 및 한 문장으로 변환
    cleaned_text = re.sub(r"\d+\.\s*", "", cleaned_text)  # "1. " 같은 숫자 제거
    cleaned_text = cleaned_text.replace("\n", " ")  # 줄바꿈을 공백으로 변경

    return cleaned_text




# 함수 실행
filtered_text = extract_clean_prevention_measures(test_results[0])
print(filtered_text)


   차량계 건설기계 작업 계획서 작성 및 검토: 작업 계획을 세우고 점검.   펌프카 안전 점검 체크리스트 사전 점검: 안전을 보장.   지반 상태 사전 점검 및 받침대 추가 설치 확인: 지반 안정성 확보.   유도자/신호수 배치: 작업 공간 관리.   타 근로자 출입금지 조치: 작업자 안전.   펌프카 작업 전 안전 교육 및 확인: 작업자 교육.   TBM 작업 내용 및 위험 포인트 공유: 작업 내부 통신.  향후 조치 계획:   재발 방지 대책 수립 및 현장 관리 철저: 지속적인 개선.   교육 및 훈련 강화: 지속적인 안전 교육.  최종 답변: 재발 방지 대책으로 차량


In [30]:
from sentence_transformers import SentenceTransformer

embedding_model_name = "jhgan/ko-sbert-sts"
embedding = SentenceTransformer(embedding_model_name)

# 문장 리스트를 입력하여 임베딩 생성
pred_embeddings = embedding.encode(test_results)
print(pred_embeddings.shape)  # (샘플 개수, 768)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.44k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/620 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

(964, 768)


In [31]:
submission = pd.read_csv(data_path+"sample_submission.csv", encoding = 'utf-8-sig')
# 최종 결과 저장
submission.iloc[:,1] = test_results
submission.iloc[:,2:] = pred_embeddings
submission.head()

# 최종 결과를 CSV로 저장
submission.to_csv('submissionV4.csv', index=False, encoding='utf-8-sig')

In [33]:
submission.to_csv('submissionV4.csv', index=False, encoding='utf-8-sig')